## HQC

Status: Currently, only BCH/REP and RS/RM are supported, and they are probably old versions. Furthermore, the decoding only works when there is only one block of data.

In [83]:
class HQC:
    
    def __init__(self, w, w_e, w_r, C_1, C_2):
        
        self.w = w
        self.w_e = w_e
        self.w_r = w_r
        
        if (C_1 == 'BCH' and C_2 == 'REP'):
            self.C = BCHREPCode(n_BCH = 1023, b = 1, D = 5, n_REP = 31)
        elif (C_1 == 'RS' and C_2 == 'RM'):
            self.C = RSRMCode(n_RS = 255, k_RS = 32, q_RS = 2**8, r_RM = 1, m_RM = 7)
        # elif (C_1 == 'RS' and C_2 == 'RS'):
            # self.C = RSRSCode(...)
        # elif (C_1 == 'RM' and C_2 == 'RM'):
            # self.C = RMRMCode(...)
        # elif (C_1 == 'GOPPA' and C_2 == 'REP'):
            # self.C = GoppaREPCode(...)
        else:
            raise ValueError('Unsupported code concatenation')
        
        
        self.n = self.C.code_n
        self.k = self.C.code_k
        self.delta = self.C.code_delta
        
        self.F = GF(2)
        self.R = PolynomialRing(self.F, 'x'); x = self.R.gen()
        self.S = QuotientRing(self.R, x**self.n - 1, 'a'); a = self.S.gen()
        
        self.pk, self.sk = self.KeyGen(self.n,self.k, self.delta, self.w, self.w_e, self.w_r)
        
        
    def KeyGen(self, n, k, delta, w, w_e, w_r):
        
        h = self.R.random_element(degree=(0,self.n))
        h = self.S(h)
        
        x = [0] * self.n
        x = self.RandomGenWithWeight(x, self.w)
        x = self.S(x)
        
        y = [0] * self.n
        y = self.RandomGenWithWeight(y, self.w)
        y = self.S(y)
        
        s = x + h * y
        
        pk = (h,s)
        sk = (x,y)
        
        return pk, sk
    
    def Encryption(self, pk, m):
        
        u = []
        v = []
        
        for i in range(0, len(m), self.k):
            u_current, v_current = self.EncryptBlock(pk, m[i:i+self.k])
            u.extend(u_current)
            v.extend(v_current)
            
        return (self.S(u), self.S(v))
            
    
    def EncryptBlock(self, pk, m):
        
        if len(m) != self.C.code_k:
            raise ValueError('wrong size of message block')
        
        e = [0] * self.n
        e = self.RandomGenWithWeight(e, self.w_e)
        e = self.S(e)
        
        r1 = [0] * self.n
        r1 = self.RandomGenWithWeight(r1, self.w_r)
        r1 = self.S(r1)
        
        r2 = [0] * self.n
        r2 = self.RandomGenWithWeight(r2, self.w_r)
        r2 = self.S(r2)
        
        errors = (self.sk[0] * r2 - r1 * self.sk[1] + e).list()
        errors_HW  = errors.count(1)
        print('HW(x * r2 - r1 * y + e) = ', errors_HW)
        print('delta = ', self.delta)
        
        u = r1 + pk[0] * r2
        
        v = self.C.Encoding(m)
        v = self.S(v)
        v = v + pk[1] * r2 + e
        
        return (u,v)
    
    def Decrypt(self, sk, c):
        
        d = c[1] - c[0] * sk[1]
        d = d.list()
        
        if len(d) != self.C.code_n:
            d = d[:-(self.n - self.C.code_n)]
        
        return self.C.Decoding(d)
        
        
    def RandomGenWithWeight(self, array, weight):

        for i in range(weight):
            cur = ZZ.random_element(0,len(array))
            while array[cur] == 1:
                cur = ZZ.random_element(0,len(array))
            
            array[cur] = 1
        
        return array

In [84]:
#code = HQC(w = 67, w_e = 77, w_r = 77, C_1 = 'RS', C_2 = 'RM')

In [85]:
code = HQC(w = 5, w_e = 5, w_r = 5, C_1 = 'BCH', C_2 = 'REP')

In [86]:
m = []
for i in range(code.C.code_k):
    elem = ZZ.random_element(0,2)
    m.append(code.C.BF(elem))

c = code.Encryption(code.pk, m)
c

HW(x * r2 - r1 * y + e) =  55
delta =  77


(a^31711 + a^31709 + a^31703 + a^31701 + a^31700 + a^31698 + a^31697 + a^31696 + a^31694 + a^31693 + a^31691 + a^31688 + a^31685 + a^31684 + a^31680 + a^31678 + a^31675 + a^31674 + a^31672 + a^31671 + a^31668 + a^31666 + a^31665 + a^31661 + a^31660 + a^31659 + a^31657 + a^31656 + a^31655 + a^31653 + a^31652 + a^31649 + a^31647 + a^31646 + a^31645 + a^31641 + a^31640 + a^31639 + a^31637 + a^31636 + a^31633 + a^31632 + a^31631 + a^31629 + a^31628 + a^31627 + a^31623 + a^31619 + a^31618 + a^31614 + a^31613 + a^31612 + a^31611 + a^31610 + a^31607 + a^31604 + a^31602 + a^31601 + a^31600 + a^31597 + a^31595 + a^31593 + a^31590 + a^31589 + a^31587 + a^31585 + a^31582 + a^31581 + a^31578 + a^31577 + a^31576 + a^31573 + a^31571 + a^31570 + a^31569 + a^31567 + a^31566 + a^31564 + a^31562 + a^31557 + a^31556 + a^31553 + a^31548 + a^31546 + a^31544 + a^31543 + a^31542 + a^31540 + a^31536 + a^31534 + a^31533 + a^31522 + a^31520 + a^31516 + a^31508 + a^31507 + a^31504 + a^31502 + a^31500 + a^31495 +

In [87]:
d = code.Decrypt(code.sk, c)
d == m

True

In [4]:
class RSRMCode:
    
    def __init__(self, n_RS, k_RS, q_RS, r_RM, m_RM, alpha = None):
        
        if not (k_RS < n_RS and n_RS <= q_RS):
            raise ValueError('Invalid RS values for n, k, and q.')
            
        self.p0, self.k_RM = is_prime_power(q_RS, get_data = True)
            
        self.n_RS = n_RS
        self.k_RS = k_RS
        self.q_RS = q_RS
        
        self.r_RM = r_RM
        self.m_RM = m_RM
        
        self.n_RM = 2**self.m_RM
        #self.k_RM = sum(binomial(self.m_RM,i) for i in range(self.r_RM + 1))
        self.d_RM = 2**(self.m_RM - self.r_RM)
        
        self.d_RS = self.n_RS - self.k_RS + 1
        
        self.code_n = self.n_RS * self.n_RM
        self.code_k = self.k_RS * self.k_RM
        self.code_d = self.d_RS * self.d_RM
        self.code_delta = floor((self.code_d - 1) / 2)
        
        self.tau_RS = floor((self.n_RS-self.k_RS)/2)
        
        # Initializing field for RS
        self.EF = GF(self.q_RS) # extension field
        self.R = PolynomialRing(self.EF, 'X')
        self.p = self.EF.primitive_element()
        
        # Constructing alpha-vector
        if not alpha:
            self.alpha = vector([self.p**i for i in range(self.n_RS)])
        else:
            self.alpha = alpha
        
        # Constructing RS generator matrix
        self.G_RS = matrix(self.EF, self.k_RS, self.n_RS, lambda i,j : self.alpha[j]**i)
        
        # Initializing field for RM
        self.BF = GF(self.p0) # base field
        
        # Constructing RM generator matrix
        self.G_RM = matrix(self.BF, self.k_RM, self.n_RM, lambda i,j : 0)
        
        self.G_RM = self.get_G(self.r_RM, self.m_RM)
        
    
    def get_G(self, r, m):
        n = 2**m
        k = sum(binomial(m,i) for i in range(r + 1))
        d = 2**(m - r)

        #if (n == k):
            # trivial code

        if (n == d):
            # repetition code
            return matrix(self.BF, k, n, lambda i,j : 1)

        elif (d == 2 and n == k + 1):
            # parity check code
            PCC = codes.ParityCheckCode(GF(self.p0), k)
            return PCC.generator_matrix()

        else:
            return block_matrix([ [self.get_G(r,m-1), self.get_G(r,m-1)], [0, self.get_G(r-1,m-1)] ])
        
        
    def Encoding(self, m, zeropad = True):
        
        m = self._IntToPol(m)
        
        rem = len(m) % self.k_RS
        
        if rem != 0:
            if zeropad:
                m.extend([self.EF(0)]*(self.k_RS-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c_RS = []
        
        # Encoding each chunk of size k
        for i in range(0, len(m), self.k_RS):
            c_RS.extend(self.EncodeChunkRS(m[i:i+self.k_RS]))
        
        
        c_RS = self._PolToInt(c_RS)
        c_RS = self._IntToByteString(c_RS)
        
        c_RSRM = []
        
        # Encoding each chunk of size k
        for i in range(0, len(c_RS), self.k_RM):
            c_RSRM.extend(self.EncodeChunkRM(c_RS[i:i+self.k_RM]))
        
        #return vector(self.F, c)
        
        return c_RSRM
        #return vector(self.BF, c_RSRM)
            
    def EncodeChunkRS(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RS:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.EF, chunk) * self.G_RS
        return c
    
    def EncodeChunkRM(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RM:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.BF, chunk) * self.G_RM

        return c
    
    def Decoding(self, r):
        
        # Check input size
        if len(r) % self.n_RM != 0:
            raise ValueError('Invalid input size')
            
        c_RM1 = []
        
        for i in range(0,len(r),self.n_RM):
            c_RM1.extend(self.CorrectErrorsRM(r[i:i+self.n_RM], self.r_RM, self.m_RM))
            
        
        c_RM2 = []
        
        for i in range(0,len(c_RM1),self.n_RM):
            c_RM2.extend(self.DecodeChunkRM(c_RM1[i:i+self.n_RM]))
            
        c_RM2 = self._ByteStringToInt(c_RM2)
        c_RM2 = self._IntToPol(c_RM2)
        
        c_RSRM = []
        
        for i in range(0,len(c_RM2),self.n_RS):
            c_RSRM.extend(self.BivariateInterpolation(c_RM2[i:i+self.n_RS]))
            
        c_RSRM = self._PolToInt(c_RSRM)
        return c_RSRM
    
    
    def BivariateInterpolation(self, chunk):
        
        if len(chunk) != self.n_RS:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.EF, self.n_RS, self.tau_RS + self.k_RS, lambda i,j : self.alpha[i]**j)
        M2 = matrix(self.EF, self.n_RS, self.tau_RS + 1, lambda i,j : chunk[i] * self.alpha[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        #if len(RK.basis()) == 0:
        #    return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R(list(sol[:self.tau_RS+self.k_RS]))
        Q1 = self.R(list(sol[self.tau_RS+self.k_RS:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.EF(0)]*(self.k_RS-len(out)))

        return out
    
    
    def CorrectErrorsRM(self, word, r, m):
        
        word = vector(self.BF, word)
        
        if (self.n_RM % len(word)) != 0:
            raise ValueError('Invalid chunk size')
        
        if (r == 0):
            return self.repetition_decode(word)
        
        if (m == r + 1):
            return 'parity check'
        
        l = len(word)
        u1 = word[:l//2]
        u2_v = word[l//2:]
        
        v_error = u1 + u2_v

        v = self.CorrectErrorsRM(v_error, r - 1, m - 1)
        u2 = u2_v + v
        
        u1_decoded = self.CorrectErrorsRM(u1, r, m - 1)
        u2_decoded = self.CorrectErrorsRM(u2, r, m - 1)
        u = 0
        
        if (u1_decoded == 'parity check' and u2_decoded == 'parity check'):
            u = self.parity_check_compare(u1, u2)
            return vector(self.BF, list(u) + list(u + v))
        else:
            return vector(self.BF, list(u1_decoded) + list(u2_decoded + v))
        
    
    
    def repetition_decode(self, word):
        
        if word.hamming_weight() >= len(word.list())//2:
            return vector(self.BF, [1 for i in range(len(word.list()))])
        elif word.hamming_weight() < len(word.list())//2:
            return vector(self.BF, [0 for i in range(len(word.list()))])
        else:
            return "decoding failure"
        
    def parity_check_compare(self, word1, word2):
        decoded1 = word1[:-1]
        decoded2 = word2[:-1]
        
        if (((decoded1.hamming_weight() % 2) == 0 and word1[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word1[-1] == 1)):
            return word1
        elif (((decoded2.hamming_weight() % 2) == 0 and word2[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word2[-1] == 1)):
            return word2
        else:
            return word1
            #return "decoding failure"
        
        
    def DecodeChunkRM(self, chunk):
        cols = self.G_RM.pivots()
        G_RM_independent = self.G_RM.matrix_from_columns(cols)
        chunk_independent = [chunk[i] for i in cols]
        
        return vector(self.BF, chunk_independent) * G_RM_independent.inverse()
    
    
    def _IntToPol(self, m):
        # Convert array of integers less than q to elements of field
        
        m_out = []
        
        for i in m:
            if not i < self.q_RS:
                raise ValueError('Invalid symbol')
            m_out.append(self.EF(ZZ(i).digits(self.p0)))
            
        return m_out
    
    def _PolToInt(self, pol_array):

        # Converts array of integers less than q to elements of Field.

        pol_out = []

        for pol in pol_array:
            if not pol in self.EF:
                raise ValueError('Invalid symbol')

            pol_out.append(ZZ(pol.polynomial().coefficients(sparse = False), base = self.p0))

        return(pol_out)

    def _IntToByteString(self, int_array):

        if(self.q_RS != 2**self.k_RM):
            raise ValueError('Invalid field size for byte representation')

        # Converts array of integers less than 256 to binary representation

        if any([(item > (2**self.k_RM - 1) or item < 0) for item in int_array]):
            raise ValueError('Invalid integer values')

        if self.k_RM == 4:
            bits = '04b'
        elif self.k_RM == 8:
            bits = '08b'
        else:
            raise ValueError('Unsupported dimension of RM')
        
        return list(''.join([format(item, bits) for item in int_array]))
    
    def _ByteStringToInt(self, byte_string):

        if(self.q_RS != 2**self.k_RM):
            raise ValueError('Invalid field size for byte representation')

        # Converts array of k_RM bit binary representations to integers

        m_out = []
        for i in range(0,len(byte_string), self.k_RM):
            current = byte_string[i:i+self.k_RM]
            m_out.append(int("".join(str(x) for x in current), 2))
            
        return m_out
        
    
    def _IntToPol(self, m):
        # Converts array of integers less than q to elements of Field.

        m_out = []

        for s in m:
            if not s < self.q_RS:
                raise ValueError('Invalid symbol')
            m_out.append(self.EF(ZZ(s).digits(self.p0)))

        return(m_out)

In [5]:
class BCHREPCode:
    
    def __init__(self, n_BCH, b, D, n_REP):
        
        self.q = 2
        self.n_BCH = n_BCH
        self.m = Mod(self.q,self.n_BCH).multiplicative_order()
        self.b = b
        self.D = D
        self.n_REP = n_REP
            
        #if (gcd(self.n_BCH, self.q) != 1):
        #    raise ValueError('Invalid input values: gcd(n,q) != 1.')
            
        if (self.n_BCH != self.q**self.m - 1):
            raise ValueError('Invalid input values: n != q^m - 1')
            
        
        # Initialize field
        self.BF = GF(self.q) # base field
        self.EF = GF(self.q**self.m) # extension field
        R.<x> = PolynomialRing(self.BF, 'x')
        self.R = R
        self.x = x
        self.R_RS = PolynomialRing(self.EF, 'X')
        self.alpha = self.EF.primitive_element()
        
        # define RS parameters
        self.k_RS = self.n_BCH - self.D + 1
        self.tau_RS = floor( (self.D - 1) / 2)
        self.alpha_RS = vector([self.alpha**i for i in range(self.n_BCH)])
        self.G_RS = matrix(self.EF, self.k_RS, self.n_BCH, lambda i,j : self.alpha_RS[j]**i)
        
        # Constructing generator matrix
        self.cosets = self.cyclotomic_cosets(self.n_BCH, self.q, self.b, self.D)
        
        self.generator_poly = self.BCH_generator_polynomial(self.x, self.alpha, self.D, self.cosets)
        
        if not (self.generator_poly.divides(self.x**self.n_BCH - 1)):
            raise ValueError('generator_poly is not a generator polynomial')
            
        self.k = self.n_BCH - self.generator_poly.degree()
        
        self.G_BCH = matrix(self.BF, self.k, self.n_BCH, lambda i,j : self.generator_poly[(j+(self.n_BCH-i)) % self.n_BCH])
        
        # Parameters for product code
        self.code_n = self.n_BCH * self.n_REP
        self.code_k = self.k
        self.code_d = self.D * self.n_REP
        self.code_delta = floor((self.code_d - 1) / 2)
        
        
    def cyclotomic_cosets(self, n, q, b, D):
        # compute cyclotomic cosets
    
        cosets = []

        for i in range(b,b+D-1):
            coset = [(i * q**j) % n for j in range(0,n-1)]
            coset = list(set(coset))
            coset.sort()
            cosets.append(coset)
        return cosets

    def minimal_polynomial(self, coset, x, alpha):
        # compute minimal polynomial from one coset
        poly = 1
        for j in range(len(coset)):
            poly *= (x - alpha**coset[j])
        return poly

    def BCH_generator_polynomial(self, x, alpha, D, cosets):
        # compute generator polynomial
        poly = self.minimal_polynomial(cosets[0], x, alpha)
        for i in range(1,D-1):
            poly = LCM(poly,self.minimal_polynomial(cosets[i],x,alpha))
        
        return poly
    
    
    def Encoding(self, message, zeropad = True):
        
        rem = len(message) % self.k
        
        if rem != 0:
            if zeropad:
                message.extend([self.BF(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunkBCHREP(message[i:i+self.k]))
        
        return c
            
    def EncodeChunkBCHREP(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        # BCH encoder
        c_BCH = vector(self.BF, chunk) * self.G_BCH
        
        # Repetition encoder
        c_BCH_REP = []
        
        for i in range(len(c_BCH)):
            for j in range(self.n_REP):
                c_BCH_REP.append(c_BCH[i])
                
        return c_BCH_REP
                
    
    def Decoding(self, c_BCH_REP):
        
        # Check input size
        if len(c_BCH_REP) % (self.code_n) != 0:
            raise ValueError('Invalid input size')
            
        c_BCH = []
        
        # Decode with REP decoder
        for i in range(0,len(c_BCH_REP),self.n_REP):
            c_BCH.append(self.DecodeChunkREP(c_BCH_REP[i:i+self.n_REP]))
        
        # Need to convert to extension field
        for i in range(len(c_BCH)):
            c_BCH[i] = self.EF(c_BCH[i])
          
        # Decode with RS decoder
        c = []
        
        for i in range(0,len(c_BCH),self.n_BCH):
            c.extend(self.DecodeChunkRS(c_BCH[i:i+self.n_BCH]))
            
        # Encode with RS encoder
        c_RS = [] 
        
        for i in range(0, len(c), self.k_RS):
            c_RS.extend(self.EncodeChunkRS(c[i:i+self.k_RS]))
        
        # Decode with BCH decoder
        if len(c_RS) % self.n_BCH != 0:
            raise ValueError('Invalid codeword size')
            
        # Need to convert back to base field
        for i in range(len(c_RS)):
            c_RS[i] = self.BF(c_RS[i])
            
        c = []
        
        cols = self.G_BCH.pivots()
        G_independent = self.G_BCH.matrix_from_columns(cols)
        
        for i in range(0,len(c_RS),self.n_BCH):
            c.extend(self.DecodeChunkBCH(c_RS[i:i+self.n_BCH], cols, G_independent))
            
        return c
    
    def DecodeChunkBCH(self, chunk, cols, G_independent):
        
        chunk_independent = [chunk[i] for i in cols]
        
        return vector(self.BF, chunk_independent) * G_independent.inverse()
    
    
    def DecodeChunkREP(self, chunk):
        if chunk.count(1) >= ceil(self.n_REP / 2):
            return 1
        else:
            return 0
    
            
    def EncodeChunkRS(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RS:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.EF, chunk) * self.G_RS
        return c
    
    def DecodeChunkRS(self, chunk):
        # Bivariate interpolation is used
        
        if len(chunk) != self.n_BCH:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.EF, self.n_BCH, self.tau_RS + self.k_RS, lambda i,j : self.alpha_RS[i]**j)
        M2 = matrix(self.EF, self.n_BCH, self.tau_RS + 1, lambda i,j : chunk[i] * self.alpha_RS[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        if len(RK.basis()) == 0:
            return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R_RS(list(sol[:self.tau_RS+self.k_RS]))
        Q1 = self.R_RS(list(sol[self.tau_RS+self.k_RS:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.EF(0)]*(self.k_RS-len(out)))

        return out